Goal of this notebook:

Perform feature selection on our dataset.

Strategy:

Iterate over each project and execute the feature selection

In [9]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore")

In [10]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])
non_features_columns.extend(["has_branch_merge_message_indicator"])

In [11]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [12]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=400, max_features=0.3, min_samples_leaf=1)

# Tree-based feature selection

Uses the feature_importances_ attribute from the Random Forest model to select the most important features. It uses the mean of the importances of all features as a threshold.

In [13]:
import importlib
importlib.reload(classifier_utils)
results_tree, attributes_record_tree = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'tree')

In [14]:
results_tree

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759,128.00,34.0,0.739,0.740,0.005
1,apache__directory-server,652,95.00,18.0,0.942,0.928,-0.015
2,android__platform_frameworks_base,2460,565.00,53.0,0.817,0.814,-0.004
3,freenet__fred,1012,133.00,47.0,0.673,0.680,0.021
4,alexo__wro4j,1368,106.00,31.0,0.583,0.585,0.004
5,apache__lucene-solr,974,123.00,42.0,0.646,0.644,-0.003
6,getrailo__railo,572,89.00,30.0,0.710,0.711,0.006
7,atlasapi__atlas,782,123.00,41.0,0.671,0.656,-0.023
8,hibernate__hibernate-orm,716,130.00,32.0,0.602,0.591,-0.018
9,CloudStack-extras__CloudStack-archive,1106,134.00,48.0,0.806,0.805,-0.001


# Recursive feature elimination:

First, the estimator is trained on the initial set of features and the importance of each feature is obtained. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
Currently we are discarding 1 feature per step, using a 5-fold-cross-validation to calculate the accuracy on each step.

In [15]:
# import importlib
# importlib.reload(classifier_utils)
# results_recursive, attributes_record_rec = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'recursive')

In [16]:
# results_recursive

# IGAR
Selects attributes based on the ranking of their information gain.
Information gain measures the ability of a feature to separate the target classes. The greater the information gain, the better its importance for classification tasks.

Information Gain = Entropy(overall) - Entropy(attribute)

The algorithm has an input value 'n' that is used to select the 'n' attributes with the greatest information gain among all attributes. In this notebook we use n = 84, which was the found in the notebook IGAR_tuning.ipynb.

In [17]:
import importlib
importlib.reload(classifier_utils)
results_IGAR, attributes_record_IGAR = classifier_utils.projects_feature_selection(projects, non_features_columns, rf, 'IGAR')

In [18]:
results_IGAR

,project,N,# attr.,# attr. fs,accuracy,accuracy_fs,improvement
0,Ramblurr__Anki-Android,759,128.00,84.0,0.739,0.743,0.015
1,apache__directory-server,652,95.00,84.0,0.942,0.933,-0.010
2,android__platform_frameworks_base,2460,565.00,84.0,0.817,0.811,-0.008
3,freenet__fred,1012,133.00,84.0,0.673,0.656,-0.025
4,alexo__wro4j,1368,106.00,84.0,0.583,0.583,-0.001
5,apache__lucene-solr,974,123.00,84.0,0.646,0.651,0.014
6,getrailo__railo,572,89.00,84.0,0.710,0.706,-0.005
7,atlasapi__atlas,782,123.00,84.0,0.671,0.675,0.012
8,hibernate__hibernate-orm,716,130.00,84.0,0.602,0.591,-0.019
9,CloudStack-extras__CloudStack-archive,1106,134.00,84.0,0.806,0.805,-0.001


## Comparison

In [19]:
df_inner = pd.merge(results_tree, results_recursive, on='project', how='inner', suffixes=('_tree', '_rec'))
df_inner_igar = results_IGAR.add_suffix("_IGAR").rename(columns={"project_IGAR": "project"})
df_inner = pd.merge(df_inner, df_inner_igar, on='project', how='inner')
df_inner.to_csv('feature_selection_comparison.csv', index=False)

accuracy_inner = df_inner.filter(regex=("project|accuracy.*")).copy()
accuracy_inner['improvement_tree'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_tree'], x['accuracy_tree']), axis=1)
accuracy_inner['improvement_rec'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_rec'], x['accuracy_rec']), axis=1)
accuracy_inner['improvement_IGAR'] = accuracy_inner.apply(lambda x: classifier_utils.get_normalized_improvement(x['accuracy_fs_IGAR'], x['accuracy_IGAR']), axis=1)
accuracy_inner = accuracy_inner.round(3)
accuracy_inner

NameError: name 'results_recursive' is not defined

In [20]:
pd.DataFrame(attributes_record_rec, columns=['project', 'attribute', 'information_gain', 'method']).to_csv('attributes_record_rec.csv', index=False)

NameError: name 'attributes_record_rec' is not defined

In [ ]:
attributes_record = []
attributes_record.extend(attributes_record_tree)
attributes_record.extend(attributes_record_rec)
attributes_record.extend(attributes_record_IGAR)
attributes_record_df = pd.DataFrame(attributes_record, columns=['project', 'attribute', 'information_gain', 'method'])
attributes_record_df.to_csv('attributes_record.csv', index=False)

In [ ]:
attributes_record = pd.read_csv('attributes_record.csv')

## Ranking of features selected by tree method

Counts in how many projects the respective feature was selected using the tree method.

In [ ]:
ranking_tree = classifier_utils.get_attribute_selection_ranking(attributes_record, 'tree')
ranking_tree.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

## Ranking of features selected by recursive method

Counts in how many projects the respective feature was selected using the recursive method.

In [ ]:
ranking_recursive = classifier_utils.get_attribute_selection_ranking(attributes_record, 'recursive')
ranking_recursive.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)

## Ranking of features selected by IGAR method

Counts in how many projects the respective feature was selected using the IGAR method. 

The information gain column is an average among all projects.

In [ ]:
import importlib
importlib.reload(classifier_utils)
ranking_IGAR = classifier_utils.get_attribute_selection_ranking(attributes_record, 'IGAR')
ranking_IGAR.sort_values(['average_information_gain', 'count_selected'], ascending=False).head(50)